In [1]:
from workshop_utils import * 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
import geopandas as gpd
import shapely
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from tqdm import tqdm_notebook as tqdm
tqdm().pandas();

# First, get extract all of the building edits for an area associated with a previous HOT task.


Getting the buildings from task https://tasks.hotosm.org/project/4690 (2018 Ebola)

```sql
SELECT
  subsaharan_africa.id, 
  geom,
  subsaharan_africa.tags, changeset, updated, valid_until, version, minor_version,
  changesets.id AS c_id, 
  changesets.tags as c_tags,
  changesets.uid,
  changesets.user
  
FROM subsaharan_africa

JOIN changesets on subsaharan_africa.changeset = changesets.id

WHERE   
  element_at(subsaharan_africa.tags, 'building') IS NOT NULL AND ST_WITHIN(
  geom , 
  ST_POLYGON('POLYGON((17.8183 0.0461, 18.9637 0.0461, 18.9637 -0.9686, 17.8183 -0.9686, 17.8183 0.0461))'))

ORDER BY updated
```

In [ ]:
building_edits = load_dataframe_from_s3('https://us-east-2.console.aws.amazon.com/athena/query/results/ba10a1bb-3347-4c40-a1ae-2f8ff63f344a/csv')

In [ ]:
place = "Mbandaka - Bikoro"

In [ ]:
building_edits.head(2)

In [ ]:
#Cast the date as date
building_edits['updated'] = building_edits.updated.progress_apply(pd.Timestamp)
building_edits['date'] = building_edits.updated.progress_apply(pd.Timestamp.date)

In [ ]:
gb_date = building_edits.groupby('date').aggregate({
    'c_id':'count',
    'id':'nunique',
    'uid':'nunique'
});
gb_date.head(2)

In [ ]:
sns.set_style('whitegrid')
ax = gb_date['c_id'].plot(figsize=(14,4))
ax.set_xlabel("Date"); ax.set_ylabel("");
ax.set_title("Edits in {}".format(place),fontsize=16);

In [ ]:
sns.set_style('whitegrid')
ax = gb_date['uid'].plot(style='.', figsize=(14,4))
ax.set_xlabel("Date"); ax.set_ylabel("");
ax.set_title("Unique editors active per day in {}".format(place),fontsize=16);

In [ ]:
from shapely import wkt

In [ ]:
building_edits['geom'] = building_edits.geom.progress_apply(wkt.loads) #Makes the geometry an actual object...

In [ ]:
objects = gpd.GeoDataFrame(building_edits, geometry='geom')

In [ ]:
objects['date'] = objects.updated.apply(lambda x: x.isoformat())

In [ ]:
from keplergl import KeplerGl 
map_1 = KeplerGl(height=600)

In [ ]:
map_1

In [ ]:
map_1.add_data(data=objects[objects.updated > datetime.datetime(2018,1,1)][
    ['geom','changeset','date','user','version','minor_version']], name='objects')


### Visualization

- Set layer blending to additive.
- fill color based on minor_version

Once you do this, buildings that glow have been edited the most.  If you zoom in on them, you can see their shapes change through each version!